In [19]:
import pandas as pd
import numpy as np
import pandas as pd
import iisignature
from tqdm import tqdm
from joblib import Parallel, delayed
import os

In [20]:
pd.read_csv('/Users/runminghuang/Documents/GitHub/capstone/datasets/sp500_final_data_5y.csv')

,symbol,date,open,high,low,close,volume,adjclose,Return,Transaction_Rate,Book_to_Market,Sector,Industry,Ticker,dividends,splits
0,BSX,2020-03-16,28.719999,29.230000,27.160000,27.549999,17232100,27.549999,NaN,0.011677,0.151956,Healthcare,Medical Devices,BSX,NaN,NaN
1,BSX,2020-03-17,28.100000,29.040001,26.340000,28.790001,18097200,28.790001,0.045009,0.012263,0.151956,Healthcare,Medical Devices,BSX,NaN,NaN
2,BSX,2020-03-18,26.980000,27.700001,25.410000,27.240000,18167500,27.240000,-0.053838,0.012310,0.151956,Healthcare,Medical Devices,BSX,NaN,NaN
3,BSX,2020-03-19,26.740000,28.870001,25.510000,28.270000,12088500,28.270000,0.037812,0.008191,0.151956,Healthcare,Medical Devices,BSX,NaN,NaN
4,BSX,2020-03-20,28.629999,28.780001,27.100000,27.240000,11222900,27.240000,-0.036434,0.007605,0.151956,Healthcare,Medical Devices,BSX,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622199,ADSK,2025-03-10,256.609985,256.929993,246.309998,248.399994,3198600,248.399994,-0.043696,0.015017,0.048139,Technology,Software - Application,ADSK,NaN,NaN
622200,ADSK,2025-03-11,247.559998,254.380005,246.949997,250.850006,2776600,250.850006,0.009863,0.013036,0.048139,Technology,Software - Application,ADSK,NaN,NaN
622201,ADSK,2025-03-12,254.500000,255.910004,250.110001,251.850006,2432100,251.850006,0.003986,0.011418,0.048139,Technology,Software - Application,ADSK,NaN,NaN
622202,ADSK,2025-03-13,251.850006,252.669998,244.850006,246.929993,1498400,246.929993,-0.019535,0.007035,0.048139,Technology,Software - Application,ADSK,NaN,NaN


In [27]:
SourceData = pd.read_csv('/Users/runminghuang/Documents/GitHub/capstone/datasets/sp500_final_data_5y.csv')[['symbol', 'date', 'volume', 'Transaction_Rate', 'Return', 'Sector', 'Industry']]
SourceData

,symbol,date,volume,Transaction_Rate,Return,Sector,Industry
0,BSX,2020-03-16,17232100,0.011677,NaN,Healthcare,Medical Devices
1,BSX,2020-03-17,18097200,0.012263,0.045009,Healthcare,Medical Devices
2,BSX,2020-03-18,18167500,0.012310,-0.053838,Healthcare,Medical Devices
3,BSX,2020-03-19,12088500,0.008191,0.037812,Healthcare,Medical Devices
4,BSX,2020-03-20,11222900,0.007605,-0.036434,Healthcare,Medical Devices
...,...,...,...,...,...,...,...
622199,ADSK,2025-03-10,3198600,0.015017,-0.043696,Technology,Software - Application
622200,ADSK,2025-03-11,2776600,0.013036,0.009863,Technology,Software - Application
622201,ADSK,2025-03-12,2432100,0.011418,0.003986,Technology,Software - Application
622202,ADSK,2025-03-13,1498400,0.007035,-0.019535,Technology,Software - Application


In [28]:
SourceData.dropna()

,symbol,date,volume,Transaction_Rate,Return,Sector,Industry
1,BSX,2020-03-17,18097200,0.012263,0.045009,Healthcare,Medical Devices
2,BSX,2020-03-18,18167500,0.012310,-0.053838,Healthcare,Medical Devices
3,BSX,2020-03-19,12088500,0.008191,0.037812,Healthcare,Medical Devices
4,BSX,2020-03-20,11222900,0.007605,-0.036434,Healthcare,Medical Devices
5,BSX,2020-03-23,17190500,0.011648,-0.051762,Healthcare,Medical Devices
...,...,...,...,...,...,...,...
622199,ADSK,2025-03-10,3198600,0.015017,-0.043696,Technology,Software - Application
622200,ADSK,2025-03-11,2776600,0.013036,0.009863,Technology,Software - Application
622201,ADSK,2025-03-12,2432100,0.011418,0.003986,Technology,Software - Application
622202,ADSK,2025-03-13,1498400,0.007035,-0.019535,Technology,Software - Application


In [31]:
def calc_signature_new(data_path, time_window=10, order=3, save=True, 
                     sig_columns=['Return', 'Transaction_Rate', 'date'], 
                     output_filename='new_sig_data'):
    """
    计算股票数据的特征签名，使用前time_window天的数据预测当天的收益率
    
    参数:
    data_path (str): 数据文件路径
    time_window (int): 用于计算签名的历史时间窗口大小
    order (int): 签名阶数
    save (bool): 是否保存结果
    sig_columns (list): 用于计算签名的列
    output_filename (str): 输出文件名
    
    返回:
    pandas.DataFrame: 包含签名特征的数据框
    """
    print(f"加载数据: {data_path}")
    if data_path.endswith('.csv'):
        df = pd.read_csv(data_path)
    elif data_path.endswith('.parquet'):
        df = pd.read_parquet(data_path)
    else:
        raise ValueError("不支持的文件格式，请使用.csv或.parquet")
    
    df = df[['symbol', 'date', 'volume', 'Transaction_Rate', 'Return', 'Sector', 'Industry']]
    print(f"数据已加载，形状: {df.shape}")
    print(f"使用时间窗口: 前{time_window}天")
    print(f"计算签名的列: {sig_columns}")
    
    # 确保日期格式正确
    df['date'] = pd.to_datetime(df['date'])
    # 对缺失值进行处理
    df = df.dropna()
    print(df.shape)
    
    # 对收益率进行对数转换 (1+x)，避免负值问题
    if 'Return' in df.columns:
        # 保存原始Return用于最终结果
        df['RET_original'] = df['Return']
        df['Return'] = np.log1p(df['Return'])
    
    # 处理交易量的尺度问题（使用对数变换）
    if 'volume' in df.columns:
        df['volume'] = np.log1p(df['volume'])
        
    # 将日期转换为数值格式(时间戳)，以便用于签名计算
    if 'date' in sig_columns:
        # 创建一个新列用于签名计算
        df['date_numeric'] = (df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
        # 调整sig_columns列表，用数值日期替换原始日期
        sig_columns = [col if col != 'date' else 'date_numeric' for col in sig_columns]
    
    # 按股票和日期排序
    df = df.sort_values(['symbol', 'date']).reset_index(drop=True)
    
    # 为每个股票创建滚动窗口
    def process_stock_group(stock_data):
        stock_symbol = stock_data['symbol'].iloc[0]
        all_signatures = []
        
        # 确保有足够的历史数据
        if len(stock_data) <= time_window:
            return []
        
        # 创建滚动窗口
        # 从索引time_window开始，这样每个窗口都有足够的历史数据
        for i in range(time_window, len(stock_data)):
            # 使用前time_window天的数据计算特征
            feature_window = stock_data.iloc[i-time_window:i]
            # 当前行（包含要预测的return）
            current_row = stock_data.iloc[i]
            
            # 提取计算签名所需数据（只包含数值型数据）
            path_data = feature_window[sig_columns].values.astype(np.float64)
            
            try:
                # 计算签名
                signature = iisignature.sig(path_data, order)
                augmented_sig = np.insert(signature, 0, 1.0)  # 增广签名
                
                # 当前行的日期（预测日）
                current_date = current_row['date']
                
                # 获取其他需要的信息
                signature_info = {
                    'symbol': stock_symbol,
                    'date': current_date,
                    'Sector': current_row['Sector'],
                    'Industry': current_row['Industry']
                }
                
                # 添加签名特征
                for j, sig_val in enumerate(augmented_sig):
                    signature_info[f'SIG_{j}'] = sig_val
                
                # 使用当前行的return作为标签
                if 'RET_original' in current_row:
                    signature_info['RET'] = current_row['RET_original']
                else:
                    signature_info['RET'] = current_row['Return']
                
                all_signatures.append(signature_info)
            except Exception as e:
                print(f"处理股票 {stock_symbol} 窗口时出错: {e}")
                continue
        
        return all_signatures
    
    # 并行处理每个股票
    print("开始计算签名...")
    all_stock_groups = list(df.groupby('symbol'))
    
    results = Parallel(n_jobs=-1)(
        delayed(process_stock_group)(group_data) 
        for _, group_data in tqdm(all_stock_groups, desc="处理股票")
    )
    
    # 合并所有结果
    all_signatures = []
    for stock_signatures in results:
        all_signatures.extend(stock_signatures)
    
    # 创建结果数据框
    signature_df = pd.DataFrame(all_signatures)
    
    # 统计结果
    print(f"计算完成！生成了 {len(signature_df)} 个签名样本")
    print(f"每个样本包含 {sum(1 for col in signature_df.columns if col.startswith('SIG_'))} 个签名特征")
    
    # 保存结果
    if save:
        output_path = f"./datasets/{output_filename}_w{time_window}_o{order}.parquet"
        os.makedirs("./datasets", exist_ok=True)
        signature_df.to_parquet(output_path, engine='pyarrow', compression='snappy')
        print(f"结果已保存至: {output_path}")
    
    return signature_df

In [32]:
# 示例使用
# 函数调用示例，您可以根据需要修改参数
window_span = 10
order_num = 3
df_signatures = calc_signature_new(
    data_path='datasets/sp500_final_data_5y.csv',
    time_window=window_span,  # 可以改为3,5,7,10等
    order=order_num,
    sig_columns=['date', 'Return', 'Transaction_Rate'],  # 计算签名的列
    output_filename=f'sig_data_SP500'
)
df_signatures

加载数据: datasets/sp500_final_data_5y.csv
数据已加载，形状: (622204, 7)
使用时间窗口: 前10天
计算签名的列: ['date', 'Return', 'Transaction_Rate']
(621704, 7)
开始计算签名...


处理股票: 100%|██████████| 480/480 [00:17<00:00, 26.91it/s]


计算完成！生成了 616904 个签名样本
每个样本包含 40 个签名特征
结果已保存至: ./datasets/sig_data_SP500_w10_o3.parquet


,symbol,date,Sector,Industry,SIG_0,SIG_1,SIG_2,SIG_3,SIG_4,SIG_5,...,SIG_31,SIG_32,SIG_33,SIG_34,SIG_35,SIG_36,SIG_37,SIG_38,SIG_39,RET
0,A,2020-03-31,Healthcare,Diagnostics & Research,1.0,13.0,-0.066790,-0.006509,84.5,0.333487,...,-0.046077,-0.000634,3.033699e-05,-0.002507,-8.258651e-06,1.496576e-06,0.000066,5.322834e-07,-4.595116e-08,-0.014449
1,A,2020-04-01,Healthcare,Diagnostics & Research,1.0,13.0,0.007987,-0.008074,84.5,-0.178081,...,-0.324018,0.001056,1.094210e-04,-0.001418,-2.184935e-06,1.909471e-07,0.000219,-5.734000e-08,-8.770939e-08,-0.037699
2,A,2020-04-02,Healthcare,Diagnostics & Research,1.0,13.0,0.007781,-0.004409,84.5,-0.496324,...,-0.173932,0.001558,-2.150058e-05,-0.000267,-4.066366e-06,-2.709191e-08,0.000104,-3.363939e-07,-1.428739e-08,0.048897
3,A,2020-04-03,Healthcare,Diagnostics & Research,1.0,13.0,0.037151,-0.004974,84.5,0.601383,...,-0.163614,-0.001920,1.390455e-05,-0.000157,-6.825228e-06,-3.126028e-08,0.000091,5.925123e-07,-2.050992e-08,-0.025868
4,A,2020-04-06,Healthcare,Diagnostics & Research,1.0,11.0,0.021402,-0.003287,60.5,-0.373816,...,-0.131180,0.000804,-2.603345e-05,0.000030,-2.432254e-06,-1.366877e-07,0.000066,-9.773366e-08,-5.916463e-09,0.055950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616899,ZTS,2025-03-10,Healthcare,Drug Manufacturers - Specialty & Generic,1.0,11.0,-0.011295,-0.000896,60.5,0.134426,...,-0.071707,-0.000210,-1.215273e-05,-0.000344,-8.341049e-07,-1.054352e-07,0.000012,7.602383e-08,-1.198678e-10,0.008394
616900,ZTS,2025-03-11,Healthcare,Drug Manufacturers - Specialty & Generic,1.0,13.0,-0.010933,-0.001830,84.5,0.024049,...,-0.177895,-0.000073,-9.052332e-06,-0.000324,-4.499345e-07,-1.338255e-07,0.000028,9.495767e-08,-1.021566e-09,-0.043132
616901,ZTS,2025-03-12,Healthcare,Drug Manufacturers - Specialty & Generic,1.0,13.0,-0.022861,0.002014,84.5,-0.640877,...,0.079560,-0.000746,7.808828e-06,-0.000085,2.082101e-06,-5.661691e-08,0.000011,-6.034040e-08,1.360837e-09,-0.016485
616902,ZTS,2025-03-13,Healthcare,Drug Manufacturers - Specialty & Generic,1.0,13.0,-0.016562,0.000292,84.5,-0.264098,...,0.167739,-0.000530,-4.970192e-05,-0.000005,1.359042e-07,1.780741e-07,0.000029,-9.956919e-08,4.163310e-12,-0.010020
